In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from crypto_research.binance.loader import get_all_trading_pairs, get_half_year_volumes, get_historical_klines,get_trading_pairs_existing_at
from datetime import datetime, timedelta
from crypto_research.influx.constants import FUTURES_BUCKET
from crypto_research.influx.saver import save_klines_influxdb

from crypto_research.influx.internal import create_bucket
from binance.enums import KLINE_INTERVAL_1DAY, KLINE_INTERVAL_1HOUR, KLINE_INTERVAL_5MINUTE

import pandas as pd

In [3]:

trading_pairs_all = await get_all_trading_pairs(usdt_only=True)
trading_pairs_filtered_history = await get_trading_pairs_existing_at(trading_pairs_all)
trading_pairs_volumes = await get_half_year_volumes(trading_pairs_filtered_history)

# Select {SELECTED_NUMBER} trading pairs with highest trading volume in 6 month
SELECTED_NUMBER = 50
selected_pairs = list(trading_pairs_volumes[:SELECTED_NUMBER].index)

100%|██████████| 129/129 [00:03<00:00, 33.62it/s]


In [6]:
# from crypto_research.influx.loader import get_klines_pairs, get_klines
# 
# selected_pairs = list(await get_klines_pairs(interval=KLINE_INTERVAL_1HOUR))

['aaveusdt',
 'unfiusdt',
 'dydxusdt',
 'kncusdt',
 'trxusdt',
 'axsusdt',
 'ltcusdt',
 'nearusdt',
 'maskusdt',
 'trbusdt',
 'galausdt',
 'bakeusdt',
 'tomousdt',
 'xrpusdt',
 'ognusdt',
 'ethusdt',
 '1000shibusdt',
 'adausdt',
 'linausdt',
 'uniusdt',
 'etcusdt',
 'ftmusdt',
 'sushiusdt',
 'solusdt',
 'runeusdt',
 'blzusdt',
 'maticusdt',
 'btcusdt',
 'mtlusdt',
 'filusdt',
 'snxusdt',
 'hbarusdt',
 'bchusdt',
 'atomusdt',
 'stmxusdt',
 'sandusdt',
 'mkrusdt',
 'xlmusdt',
 'bnbusdt',
 'crvusdt',
 'compusdt',
 'zrxusdt',
 'lptusdt',
 'dotusdt',
 'dogeusdt',
 'eosusdt',
 'linkusdt',
 'storjusdt',
 'avaxusdt',
 'wavesusdt']

In [12]:
end = datetime.now() - timedelta(days=1)
# end = end - timedelta(days=365//2 - 5)
start = end - timedelta(days=365 * 2)

# 
# start = end - timedelta(days=365 * 3)
# end = end - timedelta(days=(365 * 2 - 2))
# 
# 
# print(end, start)

2021-12-13 12:23:43.329774 2020-12-11 12:23:43.329774


In [13]:
# await create_bucket(bucket=FUTURES_BUCKET)
# 
# data_generator = await get_historical_klines(*selected_pairs, start=start, end=end, interval=KLINE_INTERVAL_5MINUTE, rate_limit_weight=2200)
# 
# errors = []
# 
# async for data_batch in data_generator:
#     await save_klines_influxdb(data_batch, interval=KLINE_INTERVAL_5MINUTE, bucket=FUTURES_BUCKET)


2023-12-12 12:23:48.762 | INFO     | crypto_research.influx.internal:create_bucket:26 - Bucket FUTURES already exists.
  8%|▊         | 440/5300 [00:40<07:30, 10.79it/s]


CancelledError: 

In [18]:
await create_bucket(bucket=FUTURES_BUCKET)

interval = '1h'
data_generator = await get_historical_klines(*selected_pairs, start=start, end=end, interval=interval, rate_limit_weight=2200)

errors = []

async for data_batch in data_generator:
    await save_klines_influxdb(data_batch, interval=interval, bucket=FUTURES_BUCKET)


2023-12-12 12:45:27.105 | INFO     | crypto_research.influx.internal:create_bucket:26 - Bucket FUTURES already exists.
100%|██████████| 450/450 [01:06<00:00,  6.77it/s]
INFO:binance:There are 6 failed requests. Retrying 1/3...
100%|██████████| 6/6 [00:04<00:00,  1.35it/s]


In [17]:

await create_bucket(bucket=FUTURES_BUCKET)

interval = KLINE_INTERVAL_1DAY
data_generator = await get_historical_klines(
    *selected_pairs, start=start, end=end, interval=interval, rate_limit_weight=2200
)

errors = []

async for data_batch in data_generator:
    await save_klines_influxdb(data_batch, interval=interval, bucket=FUTURES_BUCKET)


2023-12-12 12:41:52.288 | INFO     | crypto_research.influx.internal:create_bucket:26 - Bucket FUTURES already exists.
100%|██████████| 50/50 [00:10<00:00,  4.95it/s]


In [16]:
# datetime.fromtimestamp(1702355931999 / 1000)

datetime.datetime(2023, 12, 12, 12, 38, 51, 999000)

In [ ]:
# from crypto_research.binance.loader import _get_historical_klines
# 
# missed_data = await _get_historical_klines('xlmusdt', start=datetime(year=2022, month=4, day=15), end=datetime(year=2022, month=5, day=28), interval='1h')

#### InfluxDB read a huge amount of data is slow; instead we put it in csv files for research purposes for faster read
#### Resample data to 1h as well.

In [19]:

from crypto_research.influx.loader import get_klines_pairs, get_klines

import tqdm
from pathlib import Path
output = Path("/Users/borsden/Projects/crypto_research/data")
for interval in (KLINE_INTERVAL_1HOUR, KLINE_INTERVAL_1DAY):
    # Influxdb can not effectively handle multiple read requests out of the box, make it in a loop.
    for trading_pair in tqdm.tqdm(selected_pairs):
        data = await get_klines(pair=trading_pair, interval=interval)
        data.to_csv(output / interval / f"{trading_pair}.csv")
    # resample(data, resample_interval).to_csv(output / resample_interval / f"{trading_pair}.csv")


100%|██████████| 50/50 [00:11<00:00,  4.42it/s]


In [ ]:
data = await get_klines(interval=interval)